# 1.前処理

In [ ]:
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
# % matplotlib inline
import seaborn as sns
import japanize_matplotlib
# sns.set(font='IPAexGothic')
sns.set()
plt.rcParams['font.family']
import datetime
from numba import jit, f8, i8, b1, void,njit
import plotly.graph_objects as go
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

## 1.1差分時間を算出

In [ ]:
# testも同様に行う
train=pd.read_csv("train.csv", encoding="shift-jis")

In [ ]:
# test2["day"]=test2["時刻"].map(lambda x:x.split("-")[2].split("T")[0])
train["day"]=train["時刻"].map(lambda x:x.split("-")[2].split("T")[0])

In [ ]:

# test2["hour"]=test2["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[0])
train["hour"]=train["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[0])

In [ ]:

# test2["min"]=test2["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[1])
train["min"]=train["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[1])

In [ ]:
# test2["second"]=test2["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[2].split("Z")[0])
train["second"]=train["時刻"].map(lambda x:x.split("-")[2].split("T")[1].split(":")[2].split("Z")[0])

In [ ]:
train.to_csv("temp_train.csv",index=False)


In [ ]:
train=pd.read_csv("temp_train.csv",parse_dates=[0])
# test2=pd.read_csv("temp_test2.csv",parse_dates=[0])

In [ ]:
@jit
def make_sabuntime(dataframe):
    sabun_list=[]
    time_df=dataframe["時刻"]
    for i in range(len(time_df)):
        if(i==len(time_df)-1):
            continue
        sabun_time=time_df[i+1]-time_df[i]
        sabun_list.append(sabun_time)
    
    return sabun_list

In [ ]:
train_list=make_sabuntime(train)
train_list.append(train_list[-1])

train["sabun_time"]=train_list

In [ ]:

# 測定時間の差分を秒で数値化

@jit
def datetime_to_seconds(df):
    
    sabun_seconds_list=[]
    
    for i in range(len(df)):
        

        stos=float(str(df["sabun_time"][i]).split(" ")[2].split(":")[2])*1
        mtos=float(str(df["sabun_time"][i]).split(" ")[2].split(":")[1])*60
        htos=float(str(df["sabun_time"][i]).split(" ")[2].split(":")[0])*3600
                   
        sabun_seconds=stos+mtos+htos
        sabun_seconds_list.append(sabun_seconds)
    return sabun_seconds_list
    
    

In [ ]:
train_ss_list=datetime_to_seconds(train)
train["sabun_seconds"]=train_ss_list

In [ ]:
train.to_csv("train_ver2.csv",index=False)

## 1.2時間主軸からパイプ位置主軸

In [ ]:
train=pd.read_csv("train_ver2.csv")

# 加えたいカラム処理
train["隙間かけトーチ"]=train["隙間位置"]*train["トーチ位置"]
train["速度.m.sec."]=train["速度.m.min."]/60

# いらないからむでりーと
# del train["中央輝度.平均."],train["中央輝度.標準偏差."],train["輝度ムラ"],train["速度.m.min."]
del train["速度.m.min."]



# 差分時間の小さすぎるものを削除。これ別にあってもよかった
train=train[train["sabun_seconds"]>0.05].reset_index()



# 差分時間取得
unique_sabun_list=np.array(train["sabun_seconds"].value_counts().sort_index().index)
# 1大きい差分時間（10秒以上）取得
stop_sabun_list =unique_sabun_list[unique_sabun_list>10]
train_stop_sabun_index=[]



# 大きい差分時間のindex取得
for i ,stop_sabun in enumerate(stop_sabun_list):
#     でかい差分時間のunique数が1なので[0]にしてる
    stop_sabun_index=train[train["sabun_seconds"]==stop_sabun].index[0]
    train_stop_sabun_index.append(stop_sabun_index)
    
# DFindexを昇順にソートする、implaceはtrue
train_stop_sabun_index.sort()


In [ ]:
# 外観verをちょっといじって作ってるので、変数名がちょっと変になってる。
# true_gaikanは下流の値
# sin_ture_gaikanは外観の値

# これは渦流についての処理。外観についてもほぼ同様に行える

@jit
def calc():
    
    all_df=pd.DataFrame()
    for k ,stop_index in enumerate(train_stop_sabun_index):
        print(k)
        if(k==0):
            start_index=0
            end_index=stop_index
        else:
            start_index=train_stop_sabun_index[k-1]
            end_index=stop_index



        temp_df=train.iloc[start_index:end_index]
        temp_df=temp_df[50:].reset_index()
        print(len(temp_df))
#         if(len(temp_df)<500):
#             continue



    #     積算時間の処理

        true_gaikan_list=[]
        sin_true_gaikan_list=[]
        flag=False
        for i in range(len(temp_df)):
            sum_len=0.0
    #         この下のlen(temp_df)には特に意味はない。いっぱい回したいってだけ
            another_flag=False
            for j in range(len(temp_df)):
                if(i+j>len(temp_df)-1):
                    flag=True
                    break
                    
#                 print(i)
                time=temp_df["sabun_seconds"][i+j]
#                 print("dfdf")
                speed=temp_df["速度.m.sec."][i+j]
                delta_len=time*speed*1000
    #             長さはミリで考えることにする
    # 大体一行で13ミリ進む
    
    
                
                sum_len+=delta_len
            
#                 another_flagがTrueの時は、sin_gaikan_indexを保持し続ける
                if(another_flag==False):
                    if(sum_len>158.0-6.5):
                        sin_gaikan_index=j
                        another_flag=True

            
            
                if (sum_len>3449.7-6.5):
                    get_index=j
                    break
                if(sum_len>3449.7):
                    get_index=j
                    break


                    
            if flag:
                true_gaikan=-1
                sin_true_gaikn=-1
            else:
                true_gaikan=temp_df["渦流OK.NG"][i+j]
                sin_true_gaikn=temp_df["外観OK.NG"][i+sin_gaikan_index]

            true_gaikan_list.append(true_gaikan)
            sin_true_gaikan_list.append(sin_true_gaikn)
        print(len(true_gaikan_list))
#         print(true_gaikan_list)
        temp_df["真外観OK.NG"]=sin_true_gaikan_list
        temp_df["真渦流OK.NG"]=true_gaikan_list
#         temp_df["真外観OK.NG"]=sin_true_gaikan_list
    
#         temp_df=temp_df.assign(gaikan=true_gaikan_list)

    #     全体のDataFrameと結合
        all_df=pd.concat([all_df,temp_df])
#         print(all_df)
        print(len(all_df))
    return all_df

    # 最後のきりぬきを加える





In [ ]:
all_df=calc()

In [ ]:
# 冗長ですみません
# 最後のところを結合
start_index=train_stop_sabun_index[-1]

temp_df=train.iloc[start_index:-1]
temp_df=temp_df[50:].reset_index()
print(len(temp_df))


#     積算時間の処理

true_gaikan_list=[]
sin_true_gaikan_list=[]
flag=False
for i in range(len(temp_df)):
    sum_len=0.0
#         この下のlen(temp_df)には特に意味はない。いっぱい回したいってだけ
    another_flag=False
    for j in range(len(temp_df)):
        if(i+j>len(temp_df)-1):
            flag=True
            break

#                 print(i)
        time=temp_df["sabun_seconds"][i+j]
#                 print("dfdf")
        speed=temp_df["速度.m.sec."][i+j]
        delta_len=time*speed*1000
#             長さはミリで考えることにする
# 大体一行で13ミリ進む



        sum_len+=delta_len

#                 another_flagがTrueの時は、sin_gaikan_indexを保持し続ける
        if(another_flag==False):
            if(sum_len>158.0-6.5):
                sin_gaikan_index=j
                another_flag=True



        if (sum_len>3449.7-6.5):
            get_index=j
            break
        if(sum_len>3449.7):
            get_index=j
            break



    if flag:
        true_gaikan=-1
        sin_true_gaikn=-1
    else:
        true_gaikan=temp_df["渦流OK.NG"][i+j]
        sin_true_gaikn=temp_df["外観OK.NG"][i+sin_gaikan_index]

    true_gaikan_list.append(true_gaikan)
    sin_true_gaikan_list.append(sin_true_gaikn)
print(len(true_gaikan_list))
#         print(true_gaikan_list)
temp_df["真外観OK.NG"]=sin_true_gaikan_list
temp_df["真渦流OK.NG"]=true_gaikan_list
#         temp_df["真外観OK.NG"]=sin_true_gaikan_list


#     全体のDataFrameと結合
final_all_df=pd.concat([all_df,temp_df])

In [ ]:
final_all_df.to_csv("真karyu_ver2.csv",index=False)

２モデルの構築

In [ ]:
#!pip install japanize-matplotlib
import japanize_matplotlib
#!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
import numpy as np
import lightgbm as lgb
import time

from sklearn.model_selection import train_test_split

2.1必要なデータの抽出

In [ ]:
#渦流OK.NGを予測するモデルの構築過程を以下に示す。
#1で外観に関して処理されたデータフレームを用いれば、外観OK.NGを予測するモデルも同様に構築できる


#上で作られた"真karyu_ver2.csv"から必要な特徴量を取り出す

df_k = pd.read_csv("../真karyu_ver2.csv")
df_k = df_k.dropna()
df_k = df_k[df_k["真外観OK.NG"] != -1]
df_k = df_k[df_k["真渦流OK.NG"] != -1]
df_k = df_k.iloc[:,10:]
df_k = df_k.drop(df_k.columns[[6,7,9,10,11,12,13,14,15,16,17,19]], axis=1)

#説明変数を読み込む
X_k = df_k.iloc[:,:9]

# 目的変数を読み込む
Y_k = df_k.iloc[:,-1]


2.2予測モデルの構築

In [ ]:
# LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(
    boosting_type= 'gbdt',
    objective= 'binary',
    num_leaves= 128,
    min_child_samples= 20,
    max_depth= 9,
    n_estimators = 1000
)

# 訓練データと検証データに分ける
X_train_k, X_val_k, Y_train_k, Y_val_k = train_test_split(X_k, Y_k, stratify=Y_k, random_state=0,test_size=0.2)


#訓練データ、検証データを用いた学習
gbm.fit(X_train_k, Y_train_k, eval_set=[(X_val_k, Y_val_k)], eval_metric="binary_logloss", verbose=1, early_stopping_rounds=50 )

#最も適切なモデルでのテストデータ(X_test_k)に対する渦流OK.NGの予測
y_pred_k = gbm.predict(X_test_k, num_iteration = gbm.best_iteration_)

2.3重要度(Permutation Importance)の可視化

In [ ]:
permutation_importance = PermutationImportance(gbm, random_state=0).fit(X_train_k, Y_train_k)
eli5.show_weights(permutation_importance, feature_names=X_train_k.columns.tolist())